 <img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/01-nvidia-logo-horiz-500x200-2c50-d@2x.png" width=300>
 
# 계산과학공학회 인공지능 겨울학교 2022
# [KSCSE](http://www.cse.or.kr/) 2022  GPU Tutorial  @ High1
# Day1 - Introducion to AI 
by Hyungon Ryu | NVAITC(NVIDIA AI Tech. Center)  Korea 


![](http://www.cse.or.kr/assets/img/logo_cse.png)

# Part4 - ResNet(Residual Network)



Residual Networks
We discovered that learning in a convolutional neural network is hierarchical: each increase in the number of layers results in more complex features being learned by the layers. But despite this, it is shown empirically that there is a maximum threshold for depth with a traditional CNN model.

In a paper titled Deep Residual Learning for Image Recognition researchers from Microsoft pointed out the following:
![](https://raw.githubusercontent.com/openhackathons-org/gpubootcamp/58e1329572bebc508ba7489a9f9415d7e0592ab8/hpc_ai/ai_science_cfd/English/python/jupyter_notebook/Intro_to_DL/images/resnet.PNG)

The failure of the 56-layer CNN could be blamed on the optimization function, initialization of the network, or the famous vanishing/exploding gradient problem. Vanishing gradients are exceptionally easy to blame for this.

So what is the vanishing gradient problem? When we do backpropagation, the gradients tend to get smaller and smaller as we keep on moving backwards in the network. This means that the neurons in the earlier layers learn very slowly as compared to the neurons in the later layers in the hierarchy. The earlier layers in the network are slowest to train.

Earlier layers in the network are essential because they are responsible for learning and detecting the simple patterns and are actually the building blocks of our network. If they give improper and inaccurate results, we can't expect the next layers and the complete network to perform nicely and produce accurate results.

The problem of training very deep networks has been alleviated with the introduction of a new neural network layer — the residual block.

Optional - The Degradation Problem

The degradation problem suggests that solvers might have difficulties in approximating identity mappings by multiple nonlinear layers without the residual learning reformulation.

Let us consider network A having  layers and network B having  layers. Supposing that , if network A performs poorly relative to network B, one might argue that if network A had mapped an identity function for the first  layers, then it would have performed on par with network B. But it doesn't do that due to the vanishing gradient problem, so when we use residual networks, the network gets the input along with learning on the residual, and if the input function was appropriate, it could quickly change the weights of the residual function to be zero.



## Residual Blocks
In a residual block the activation of a layer is fast-forwarded to a deeper layer in the neural network. Residual blocks help in the flow of information from the initial layers to the final layers. This is done by the introduction of skip connections, as seen in the image below.

Let us consider $H(x)$  as an underlying mapping to be fit by a few stacked layers (not necessarily the entire net), with  denoting the inputs to the first of these layers. If one hypothesizes that multiple nonlinear layers can asymptotically approximate complicated functions, then it is equivalent to hypothesize that they can asymptotically approximate the residual functions, i.e.,  (assuming that the input and output are of the same dimensions).

So rather than expecting the stacked layers to approximate $H(x)$, we explicitly let these layers approximate a residual function.

$F(x) = H(x) − x$.

The original function thus becomes $F(x)+x$ .

Although both models should be able to approximate the desired functions asymptotically, the ease of learning might be different. This reformulation is motivated by the counterintuitive phenomena about the degradation problem. As we discussed above, if the added layers can be constructed as identity mappings, a deeper model should have training error no greater than its shallow counterpart.
![](https://raw.githubusercontent.com/openhackathons-org/gpubootcamp/58e1329572bebc508ba7489a9f9415d7e0592ab8/hpc_ai/ai_science_cfd/English/python/jupyter_notebook/Intro_to_DL/images/resblock.PNG)

With this, increasing the number of layers improves accuracy. Here are some results from the paper:
![](https://raw.githubusercontent.com/openhackathons-org/gpubootcamp/58e1329572bebc508ba7489a9f9415d7e0592ab8/hpc_ai/ai_science_cfd/English/python/jupyter_notebook/Intro_to_DL/images/stats.png)

Now let us see how to write a residual block in Keras.

In [ ]:
# Import Necessary Libraries

from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# Let's Import the Dataset
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# Defining class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# Printing the Size of our Dataset

# Print array size of training dataset
print("Size of Training Images: " + str(train_images.shape))
# Print array size of labels
print("Size of Training Labels: " + str(train_labels.shape))

# Print array size of test dataset
print("Size of Test Images: " + str(test_images.shape))
# Print array size of labels
print("Size of Test Labels: " + str(test_labels.shape))

# Let's see how our outputs look
print("Training Set Labels: " + str(train_labels))
# Data in the test dataset
print("Test Set Labels: " + str(test_labels))

In [ ]:
# Data Preprocessing 

plt.figure()
plt.imshow(train_images[0], cmap='gray')
plt.grid(False)
plt.show()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Let's print to verify whether the data is of the correct format.
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
train_images = train_images.reshape(train_images.shape[0], w, h, 1)
test_images = test_images.reshape(test_images.shape[0], w, h, 1)

# Model
Let's build a resnet model with Keras and train it.

We will be using two kinds of residual blocks:

 - Identity Block
 - Convolution Block

 ## Identity Block
In the identity block we have a skip connection with no change in input, paired with a standard set of convolutional layers.

![](https://raw.githubusercontent.com/openhackathons-org/gpubootcamp/58e1329572bebc508ba7489a9f9415d7e0592ab8/hpc_ai/ai_science_cfd/English/python/jupyter_notebook/Intro_to_DL/images/identity.png)

## Convolution Block
The convolution block is very similar to the identity block, but there is a convolutional layer in the skip-connection path just to change the dimension such that the dimension of the input and output matches.
![](https://raw.githubusercontent.com/openhackathons-org/gpubootcamp/58e1329572bebc508ba7489a9f9415d7e0592ab8/hpc_ai/ai_science_cfd/English/python/jupyter_notebook/Intro_to_DL/images/conv.png)

Let's start building the identity block:

In [ ]:
def identity_block(X, f, filters, stage, block):
    
    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve filters
    F1, F2, F3 = filters
    
    # A path is a block of conv followed by batch normalization and activation
    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

Convolution Block
Notice the only change we need to do is add a convolution and batch normalisation for the input data to match the output dimension.

This can be done by adding the following lines :


```
##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)
```    

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):

    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def ResNet(input_shape = (28, 28, 1), classes = 10):

    # Define the input as a tensor with shape input_shape
    X_input = Input(shape=input_shape)


    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (3, 3), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2,2), padding='same')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)


    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet')

    return model

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()
model = ResNet(input_shape = (28, 28, 1), classes = 10)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs = 10, batch_size = 128)

In [ ]:
plt.plot(hist.history['loss'])

In [ ]:
plt.plot(hist.history['accuracy' ])
plt.plot(hist.history[ 'val_accuracy'])
plt.show()

## Making Predictions

In [ ]:
# Making predictions from the test_images

predictions = model.predict(test_images)

In [ ]:
# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
train_images = train_images.reshape(train_images.shape[0], w, h)
test_images = test_images.reshape(test_images.shape[0], w, h)

# Helper functions to plot images 
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

## Conclusion

Running all of our models for five epochs, and check the result and compare them

In [ ]:
# Let's Import the Dataset
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# Defining class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
# Printing the Size of our Dataset

# Print array size of training dataset
print("Size of Training Images: " + str(train_images.shape))
# Print array size of labels
print("Size of Training Labels: " + str(train_labels.shape))

# Print array size of test dataset
print("Size of Test Images: " + str(test_images.shape))
# Print array size of labels
print("Size of Test Labels: " + str(test_labels.shape))

# Let's see how our outputs look
print("Training Set Labels: " + str(train_labels))
# Data in the test dataset
print("Test Set Labels: " + str(test_labels))

In [ ]:
# Data Preprocessing 

plt.figure()
plt.imshow(train_images[0], cmap='gray')
plt.grid(False)
plt.show()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Let's print to verify whether the data is of the correct format.
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28
train_images = train_images.reshape(train_images.shape[0], w, h, 1)
test_images = test_images.reshape(test_images.shape[0], w, h, 1)

# end of jupyter part4
### Navigation  |[Part1-reg](part1_LR.ipynb) |  [Part2-MLP](part2_MLP.ipynb) |  [Part3-CNN](part3_CNN.ipynb) |  [Part4-ResNet](part4_resnet.ipynb) | [Part5-MLP Mixer](part5_Mixer.ipynb) |

<img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/01-nvidia-logo-horiz-500x200-2c50-d@2x.png" width=300>